In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Lets check your GPU
if tf.config.list_logical_devices('GPU') != []:
    print("LET'S DO THISS!!! :DDDD")
else: 
    print("Oh dude... D:")

LET'S DO THISS!!! :DDDD


## Load data

In [3]:
with open('data/input.txt') as f:
    content = f.read().lower()

In [4]:
print(content[:100])

first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you


In [5]:
all_characters = sorted(set(content.lower()))
all_tokens = all_characters + ['<PAD>', '<UNK>']
"".join(all_characters), "".join(all_tokens)

("\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz",
 "\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz<PAD><UNK>")

## Preprocess Data

Lets preprocess data. The key idea is cut all data in windows.

In [6]:
np.random.seed(42)

input_data_size = 128
window_size = input_data_size + 1
n_total_char = len(content)
n_total_tokens = len(all_tokens)

start_window_list = range(0, n_total_char - window_size)

start_window_list = np.random.choice(start_window_list, size=200_000, replace=False)

start_window_list[:10], len(start_window_list)

(array([1109478,  305053,  374053,   43191,  132750,  947160,  196038,
         793705,  184852,  765738]),
 200000)

In [7]:
# Create encoder
char_to_index = {char: index for index, char in enumerate(all_tokens)}

# Create decoder
index_to_char = np.array(all_tokens)

def encode(text):
    return np.array([char_to_index.get(char, char_to_index['<UNK>']) for char in text])

def decode(vector):
    return "".join(index_to_char[vector])

In [8]:
windows_list = np.array([(content[i:i+window_size - 1], content[i+window_size - 1:i+window_size]) for i in start_window_list])
windows_list[:10]

array([['ses too.\n\nantonio:\nwhat impossible matter will he make easy next?\n\nsebastian:\ni think he will carry this island home in his pock',
        'e'],
       [' will not sup to-night.\ngive me some ink and paper.\nwhat, is my beaver easier than it was?\nand all my armour laid into my tent?\n',
        '\n'],
       ['ds confess,\nbecause my power is weak and all ill left:\nbut if i could, by him that gave me life,\ni would attach you all and make',
        ' '],
       ["n.\n\nvolumnia:\ni know not where to turn: o, welcome home:\nand welcome, general: and ye're welcome all.\n\nmenenius:\na hundred thous",
        'a'],
       [" here's\nwater to quench it. i was hardly moved to come to\nthee; but being assured none but myself could move\nthee, i have been b",
        'l'],
       ['terly remorse confutes mine honour,\nand i did yield to him: but the next morn betimes,\nhis purpose surfeiting, he sends a warran',
        't'],
       [":\n'zounds, he dies: i had forgot the rew

In [9]:
X, y = [], []
for x in windows_list:
    X.append(encode(x[0]))
    y.append(encode(x[1]))
X = np.array(X)
y = np.array(y)

n_all_char = len(all_characters)

X = X
y = y 

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:

X_train.shape

(160000, 128)

In [240]:
X_train[:, :, np.newaxis].shape

(160000, 128, 1)

## Create Model

In [248]:
rnn_stack = keras.Sequential([
    keras.layers.Embedding(input_dim=n_all_char, output_dim=16),
    keras.layers.GRU(128),
])

dense_stack = keras.Sequential([
    keras.layers.Dense(n_all_char, activation='softmax')
])

model = keras.Sequential([
    rnn_stack,
    dense_stack
])

In [249]:
X_train[:10][:, :, np.newaxis].shape

(10, 128, 1)

In [250]:
dense_stack(rnn_stack(X_train[:10]))

<tf.Tensor: shape=(10, 39), dtype=float32, numpy=
array([[0.02574085, 0.02552984, 0.02565905, 0.02564526, 0.02569479,
        0.02558819, 0.02570878, 0.02578306, 0.02558905, 0.02572094,
        0.02578873, 0.02553393, 0.02550666, 0.02573321, 0.02552141,
        0.02565779, 0.02545676, 0.0254726 , 0.02564199, 0.02562056,
        0.02565015, 0.02534459, 0.02587151, 0.02567762, 0.02543338,
        0.02578236, 0.02561135, 0.02567596, 0.0255293 , 0.02597448,
        0.02584625, 0.02576393, 0.02570429, 0.02537828, 0.02557098,
        0.02555002, 0.02555855, 0.0258151 , 0.0256684 ],
       [0.02557015, 0.02556382, 0.0258853 , 0.02547315, 0.02572223,
        0.02554747, 0.02560454, 0.02581258, 0.02542062, 0.02568618,
        0.0255821 , 0.02575505, 0.02557503, 0.02544116, 0.02553587,
        0.02563226, 0.02572292, 0.02530951, 0.02562159, 0.0256429 ,
        0.02536375, 0.02561276, 0.02573848, 0.02566213, 0.0255566 ,
        0.02573719, 0.02575745, 0.02595556, 0.02578335, 0.02610767,
        0

In [251]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='nadam', 
              metrics=['accuracy'])

In [252]:
model.summary()

Model: "sequential_67"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_65 (Sequential)  (None, 128)               56688     
                                                                 
 sequential_66 (Sequential)  (None, 39)                5031      
                                                                 
Total params: 61,719
Trainable params: 61,719
Non-trainable params: 0
_________________________________________________________________


In [253]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
4000/4000 [==============================] - 101s 25ms/step - loss: 2.3244 - accuracy: 0.3262 - val_loss: 2.0600 - val_accuracy: 0.3938
Epoch 2/10
4000/4000 [==============================] - 136s 34ms/step - loss: 1.9636 - accuracy: 0.4196 - val_loss: 1.8830 - val_accuracy: 0.4427
Epoch 3/10
4000/4000 [==============================] - 140s 35ms/step - loss: 1.8084 - accuracy: 0.4627 - val_loss: 1.7865 - val_accuracy: 0.4700
Epoch 4/10
4000/4000 [==============================] - 121s 30ms/step - loss: 1.7111 - accuracy: 0.4876 - val_loss: 1.7407 - val_accuracy: 0.4819
Epoch 5/10
4000/4000 [==============================] - 86s 22ms/step - loss: 1.6424 - accuracy: 0.5051 - val_loss: 1.7065 - val_accuracy: 0.4915
Epoch 6/10
4000/4000 [==============================] - 102s 26ms/step - loss: 1.5928 - accuracy: 0.5172 - val_loss: 1.6850 - val_accuracy: 0.5007
Epoch 7/10
4000/4000 [==============================] - 92s 23ms/step - loss: 1.5520 - accuracy: 0.5293 - val_loss: 1.6

## Fake text

In [12]:
get_pretrained_model = True

if get_pretrained_model:
    model = keras.models.load_model('Saved Models/model_shakespeare.h5')


In [13]:

input_text = 'to be or not '
encoded_text = encode(input_text)
prediction = model.predict(encoded_text[np.newaxis, :], verbose=0)
decode(encoded_text), decode(np.argmax(prediction))

('to be or not ', 't')

In [22]:
input_text = 'to be or not '
encoded_text = encode(input_text)
print(input_text, end='', flush=True)
for i in range(128):
    
    prediction = model.predict(encoded_text[np.newaxis, :], verbose=0)
    encoded_text = np.append(encoded_text, np.argmax(prediction))
    decoded_text = decode(encoded_text)
    print(decoded_text[-1], end='', flush=True)

to be or not too

king richard iii:
and the pater to the patient that the markent here in his consent
of the patient that the marker and her 

In [23]:
# model.save('Saved Models/model_shakespeare.h5')